In [1]:
%load_ext autoreload
%autoreload 2

General questions:
* for data types that include a varint, should I add a `count` field when deserializing?
* naming: 
    * should `msg` always be a dictionary? 
    * what variable should represent the bytestring?
- should i nest `net_addr` and `services`?

In [3]:
from lib import *

In [ ]:
def deserialize_address(stream, timestamp):
    r = {}
    if timestamp:
        r["time"] = little_endian_to_int(stream.read(4))
    r["services"] = stream.read(8)
    r["ip"] = bytes_to_ip(stream.read(16))
    r["port"] = big_endian_to_int(stream.read(2))
    return r


def deserialize_version_payload(stream):
    r = {}
    r["version"] = little_endian_to_int(stream.read(4))
    r["services"] = little_endian_to_int(stream.read(8))
    r["timestamp"] = little_endian_to_int(stream.read(8))
    
    # r["receiver_address"] = deserialize_address(stream, timestamp=False)
    r["receiver_services"] = little_endian_to_int(stream.read(8))
    r["receiver_ip"] = bytes_to_ip(stream.read(16))
    r["receiver_port"] = big_endian_to_int(stream.read(2))
    
    # r["sender_address"] = deserialize_address(stream, timestamp=False)
    r["sender_services"] = little_endian_to_int(stream.read(8))    
    r["sender_ip"] = bytes_to_ip(stream.read(16))
    r["sender_port"] = big_endian_to_int(stream.read(2))
    
    r["nonce"] = little_endian_to_int(stream.read(8))
    r["user_agent"] = stream.read(read_varint(stream))
    r["latest_block"] = little_endian_to_int(stream.read(4))
    r["relay"] = little_endian_to_int(stream.read(1))
    return r


def deserialize_verack_payload(stream):
    return {}

def deserialize_addr_payload(stream):
    r = {}
    count = read_varint(stream)
    r["addresses"] = [read_address(stream) for _ in range(count)]
    return r

def deserialize_payload(command, stream):
    command_to_handler = {
        "version": deserialize_version_payload,
        "verack": deserialize_empty_payload,
        "addr": deserialize_addr_payload,
    }
    handler = command_to_handler[command]
    return handler(stream)
    
    
def deserialize_message(stream):
    """
    nest payload attributes
    """
    magic = stream.deserialize(4)
    if magic != NETWORK_MAGIC:
        raise Exception(f'Magic is wrong: {magic}')
    command = stream.deserialize(12)
    command = command.strip(b'\x00')
    payload_length = int.from_bytes(stream.deserialize(4), 'little')
    checksum = stream.deserialize(4)
    raw_payload = stream.deserialize(payload_length)
    calculated_checksum = double_sha256(raw_payload)[:4]
    if calculated_checksum != checksum:
        raise Exception('Checksum does not match')
    payload = deserialize_payload(command, BytesIO(raw_payload))
    return {
        "command": command,
        "payload": payload,
    }

In [ ]:
def deserialize_message(stream):
    """
    command and payload attributes at top level
    """
    msg = {}
    magic = stream.deserialize(4)
    if magic != NETWORK_MAGIC:
        raise Exception(f'Magic is wrong: {magic}')
    command = stream.deserialize(12)
    msg["command"] = command.strip(b'\x00')
    payload_length = int.from_bytes(stream.deserialize(4), 'little')
    checksum = stream.deserialize(4)
    raw_payload = stream.deserialize(payload_length)
    calculated_checksum = double_sha256(raw_payload)[:4]
    if calculated_checksum != checksum:
        raise Exception('Checksum does not match')
    payload = deserialize_payload(command, BytesIO(raw_payload))
    msg.update(payload)
    return msg

I tend to prefer this second one, especially if we avoid nexting the "node observations" ... 

It's much better than having "network envelopes" and "payloads / messages". One top level function shoudl hadnle everything



In [ ]:
def serialize_version_payload(
        version=70015, services=0, timestamp=None,
        receiver_services=0,
        receiver_ip='0.0.0.0', receiver_port=8333,
        sender_services=0,
        sender_ip='0.0.0.0', sender_port=8333,
        nonce=None, user_agent=b'/buidl-bootcamp/',
        latest_block=0, relay=True):
    if timestamp is None:
        timestamp = int(time.time())
    if nonce is None:
        nonce = randint(0, 2**64)
    result = int_to_little_endian(version, 4)
    result += int_to_little_endian(services, 8)
    result += int_to_little_endian(timestamp, 8)
    result += int_to_little_endian(receiver_services, 8)
    result += ip_to_bytes(receiver_ip)
    result += int_to_big_endian(receiver_port, 2)
    result += int_to_little_endian(sender_services, 8)
    result += ip_to_bytes(sender_ip)
    result += int_to_little_endian(sender_port, 2)
    result += int_to_little_endian(nonce, 8)
    result += encode_varint(len(user_agent))
    result += user_agent
    result += int_to_little_endian(latest_block, 4)
    result += int_to_little_endian(int(relay), 1)
    return result


def serialize_empty_payload(**kwargs):
    return b""

def serialize_payload(**kwargs):
    command_to_handler = {
        "version": serialize_version_payload,
        "verack": serialize_empty_payload,
        "getaddr": serialize_empty_payload,
    }
    handler = command_to_handler[kwargs['command']]
    return handler(**kwargs)

def serialize_msg(**kwargs):
    result = NETWORK_MAGIC
    result += kwargs['command'] + b'\x00' * (12 - len(command))
    payload = serialize_payload(**kwargs)  # right now 'command' will blow this up ...
    result += int_to_little_endian(len(payload), 4)
    result += double_sha256(payload)[:4]
    result += payload
    return result


In [8]:
def inner():
    return -1

def outer(**kwargs):
    return inner(**kwargs)
    
outer()

-1

In [17]:
from proto import serialize_msg
# FILL THESE IN!
address = ("35.198.151.21", 8333)

sock = socket.create_connection(address)
stream = sock.makefile("rb")

msg = serialize_msg(command=b'version')
sock.send(msg)

response = stream.read(10)

print("response: ", response)

response:  b'\xf9\xbe\xb4\xd9versio'
